## Preprocessing the Data

In [1]:
import pandas as pd
import numpy as np

### Step 1: Filter Breast Cancer Cell Lines (Cell_Lines_Details.csv)

#### Goal is to get only the breast cancer lines
- We need to get the COSMIC ID from one file and match it from the ones in the other file that have the site as 'breast'

In [ ]:
df_details = pd.read_excel('../data/raw_data/Cell_Lines_Details.xlsx')
## COSMIC ID sheet, if needed includes site and histology
## df_COSMIC = pd.read_excel('../data/raw_data/Cell_Lines_Details.xlsx', sheet_name=1)

C:\Users\Arnav Harsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
df_details.head(5)

,Sample Name,COSMIC identifier,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
0,A253,906794.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,NaN,MSS/MSI-L,D/F12,Adherent
1,BB30-HNC,753531.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
2,BB49-HNC,753532.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
3,BHY,753535.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
4,BICR10,1290724.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent


In [9]:
filtered_details_df = df_details[df_details['GDSC\nTissue\ndescriptor 2'] == 'breast']
filtered_details_df.head(5)
print(len(filtered_details_df))

52


N is only 52, therefore we only have 52 samples.
Since we are focusing on just breast cancer this helps us be more consistant and let our results be more interpretable.

### Step 2: Filter rows of GDSC2_FITTED_DOSE_RESPONSE.csv to only contain those cell lines with matching COSMIC ID from filtered_details_df